In [1]:
# from google.colab import drive
# drive.mount("/content/drive")

# BASE_DIR = "./drive/MyDrive/ml-project/dl-sprint"

In [2]:
import cv2
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [3]:
image = cv2.imread("./dls-train-balanced/common_voice_bn_30614357.png")
print(image.shape)

(374, 500, 3)


In [4]:
#@title Model Creation
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation="relu", input_shape=image.shape))
model.add(MaxPool2D(strides=2))
model.add(Conv2D(filters=48, kernel_size=(5,5), padding="valid", activation="relu"))
model.add(MaxPool2D(strides=2))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dense(84, activation="relu"))
model.add(Dense(2, activation="softmax"))

model.build()
model.summary()

adam = Adam(learning_rate=0.0001)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=adam)

print("Model built and optimizer added")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 370, 496, 32)      2432      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 185, 248, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 181, 244, 48)      38448     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 90, 122, 48)      0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 527040)            0         
                                                                 
 dense (Dense)               (None, 256)               1

In [5]:
df = pd.read_csv("./dls-train-balanced.csv")

X = df["imgname"]
y = df["label"]

X_imgname = X.values
X_img = []
lim = len(X)

for i in tqdm(range(lim)):
  X_img.append(cv2.imread(f"./dls-train-balanced/{X_imgname[i]}"))

print(f"{lim} Image Loaded into Memory")

# X = [ cv2.imread(f"{BASE_DIR}/dls-clean-8001/{filename}") for filename in X ]
X = np.array(X_img)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

y_train = keras.utils.to_categorical(y_train, num_classes=2)
y_test = keras.utils.to_categorical(y_test, num_classes=2)

print("Dataset and Image Loaded")

print("X_train:", len(X_train))
print("y_train:", len(y_train))

print("X_test:",  len(X_test))
print("y_test:",  len(y_test))

print("Dataset and Image Loaded")

100%|██████████| 19130/19130 [00:00<00:00, 39930.17it/s]

19130 Image Loaded into Memory
Dataset and Image Loaded
X_train: 15304
y_train: 15304
X_test: 3826
y_test: 3826
Dataset and Image Loaded


In [6]:
#@title Model Training
model.fit(x=X_train, y=y_train, batch_size=32, epochs=10)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type NoneType).

In [ ]:
#@title Model Prediction
y_pred = model.predict(X_test)
y_pred = np.round(y_pred)

y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

conf_mat = confusion_matrix(y_test, y_pred)
conf_dis = ConfusionMatrixDisplay(conf_mat)
conf_dis.plot()
plt.show()